In [ ]:
!pip install --quiet git+https://github.com/chrisliu/lacc2023-module3-ml

In [ ]:
from lacc.maze import (
    Maze,
    MazeBoard,
    draw_maze,
    animate_maze,
    LEFT,
    RIGHT,
    UP,
    DOWN,
)
import lacc.maze.instructor as instructor

In [ ]:
example_maze = MazeBoard([
    '######## ',
    '#   #  # ',
    '# # # ###',
    '###S    #',
    '#G# ### #',
    '# #  #  #',
    '# ## # ##',
    '#      # ',
    '######## ',
])

# What is a Maze?

In [ ]:
example_maze.reset()
draw_maze(example_maze)

## We want to find the solution from the start to the goal!
* Start = Green
* Goal = Red

In [ ]:
instructor.solve_maze(example_maze)
animate_maze(example_maze)

## Question: How do we translate a real-world problem into something a computer can solve?

Student Answers:
* TBD

## Three Objectives
1. State Representation: how do we represent a maze?
2. Transition Between States: what moves can we make in a maze?
3. Starting & Goal State: where do we start and finish a maze?

# Objective 1: Representing a Maze
* For this assignment, we prepared a `MazeBoard` class.

In [ ]:
example_maze = MazeBoard([
    '######## ',
    '#   #  # ',
    '# # # ###',
    '###S    #',
    '#G# ### #',
    '# #  #  #',
    '# ## # ##',
    '#      # ',
    '######## ',
])

In [ ]:
example_maze

> It functions like a 2D array, with some special fields to help us do the animations.

In [ ]:
example_maze[0][0]

In [ ]:
example_maze[0][0] == '#'

In [ ]:
example_maze[0][0] == Maze.WALL # For better code readability.

In [ ]:
# Maze.WALL  == '#'
# Maze.START == 'S'
# Maze.GOAL  == 'G'
# Maze.FLOOR == ' '

# Objective 3: Where do we start solving the maze?

Let's say the coordinate where we see `Maze.START` (or `S`).

This could matter for mazes with multiple goals, or even multiple starting points!

## Class Assignment: Find Starting Point (5 minutes)
Let's review some basics programming principles from the previous modules!

In [ ]:
def find_start(maze):
    # Return the row and column we should start the maze.

    for r, row in enumerate(maze):
        for c, val in enumerate(row):
            if val == Maze.START:
                return r, c
    return None

In [ ]:
draw_maze(example_maze, grid_px=30)

In [ ]:
find_start(example_maze)

# Objective 1: Keeping Track of the Player
Now we know where top start, we need to keep track of our current location.
* We'll use the position variable `me` to keep track of the current row and column.

In [ ]:
me = find_start(example_maze)
me

In [ ]:
example_maze.highlight(me)
draw_maze(example_maze)
example_maze.reset()

# Objective 2: How could we explore the maze?
What actions can we perform from where we're at?

In [ ]:
example_maze.explore(me)

...

draw_maze(example_maze, only_visible=True)
example_maze.reset()

## Class Assignment: Can we move to the right? (5 minutes)
Let's check if we can move to the right of `me`.

In [ ]:
# Maze.WALL  == '#'
# Maze.START == 'S'
# Maze.GOAL  == 'G'
# Maze.FLOOR == ' '

def is_wall(maze, pos):
    # Is the element at this position a wall?
    r, c = pos
    return maze[r][c] == Maze.WALL

def on_right(me):
    # What is the position to the right of me?
    r, c = me
    return r, c + 1

def can_move_right(maze, me):
    # Can we move to the right of me?
    pos = on_right(me)
    return not is_wall(maze, me)

In [ ]:
right = on_right(me)

In [ ]:
is_wall(example_maze, right)

In [ ]:
can_move_right(example_maze, me)

# Class Assignment: Can we move in a particular direction? (10 minutes)
Let's check if we can move in some direction.

In [ ]:
# Directions:
print(UP)
print(DOWN)
print(LEFT)
print(RIGHT)

def on_my(me, direction):
    # What is the position to the <direction> of me?
    r, c = me
    if direction == UP:
        return r - 1, c
    elif direction == DOWN:
        return r + 1, c
    elif direction == LEFT:
        return r, c - 1
    elif direction == RIGHT:
        return r, c + 1
    return None

def can_move(maze, me, direction):
    # Can we move to the <direction> of me?
    pos = on_my(me, direction)
    return not is_wall(pos)

In [ ]:
pos = on_my(me, RIGHT)
can_move(example_maze, pos, RIGHT)

In [ ]:
for direction in [UP, DOWN, LEFT, RIGHT]:
    pos = on_my(me, direction)
    available = can_move(example_maze, pos, direction)
    print(pos, available)

## Class Assignment: Try to make a sequence of moves if possible. (10 minutes)

In [ ]:
def try_move_me(maze, me, moves):
    # Perform all the <moves> if possible, returning the final position.
    # Otherwise, return None
    for direction in moves:
        me = on_my(me, direction)
    return me

In [ ]:
moves = [RIGHT, RIGHT, RIGHT, RIGHT, 
         DOWN, DOWN, 
         LEFT, 
         DOWN, DOWN, 
         LEFT, LEFT, 
         UP, UP, 
         LEFT, 
         UP]

me = find_start(example_maze)
try_move_me(example_maze, me, moves)

# Are we missing state information?

In [ ]:
example_maze.reset()
me = find_start(example_maze)
moves = [RIGHT, RIGHT, RIGHT, RIGHT, 
         DOWN, DOWN, 
         LEFT, 
         DOWN, DOWN, 
         LEFT, LEFT, 
         UP, UP, 
         LEFT, 
         UP]
instructor.make_moves(example_maze, me, moves)
animate_maze(example_maze)

> We need some way to keep track of where we've been.

## Let's define explored areas using a `set`!

### Set Basics
A set is a collection of *unique* data.

In [ ]:
s = set()
s

In [ ]:
pos = (0, 0)

In [ ]:
pos in s

In [ ]:
s.add(pos)
pos in s

In [ ]:
explored = set()

# Exploration Strategies


## Depth-First Search (DFS)
![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7f/Depth-First-Search.gif/220px-Depth-First-Search.gif)

1. Begin with the start state.
2. Check the latest node.
    * If it's the goal state.
    * If not, repeat 2 for each of its children.
3. If all the children don't match the goal, *backtrack*.

### Class Assignment: Find first occurrence in a tree using DFS. (10 minutes)
We want to search through the tree `tree` and find the first occurrence of `2023`.

In [ ]:
tree    = [2022, 2021, 2023, 2019, 2023, 2002, 1999]
# index =     0,    1,    2,    3,    4,    5,    6

The first state is `tree[0]`. We want to explore the next states.

The children of `tree[i]` are `tree[2 * i + 1]` and `tree[2 * i + 2]`.

In [ ]:
def get_children_idxs(i):
    # Generate the next actions.
    # Return the children indices.
    idx1 = 2 * i + 1
    idx2 = 2 * i + 2
    return [id1, idx2]

def is_goal(tree, i, goal):
    # Check goal state.
    # Returns if the value at tree[i] is the goal.
    return tree[i] == goal

def first_dfs(tree, i, goal):
    # Returns the first index where the goal is encountered.
    # 1. Begin with the start state.
    # 2. Check the latest node.
    #     * If it's the goal state.
    #     * If not, repeat 2 for each of its children.
    # 3. If all the children don't match the goal, return None.
    
    if is_goal(tree, i, goal):
        return i
    
    for child_i in get_children_idxs(i):
        match_i = first_dfs(tree, child_i, goal)
        if match_i is not None:
            return match_i
    
    return None

In [ ]:
start = 0
goal = 2023
first_dfs(tree, start, goal) # Should return 4.

### Bonus Challenge: Find the last occurence in a tree using DFS. (15 minutes)
In DFS order, we want to find the last time we see `2023`.

In [ ]:
def last_dfs(tree, i, goal):
    # Returns the last index where goal is encountered.
    # Hint: we need to search through the entire tree.
        
    for child_i in reversed(get_children_idxs(i)):
        match_i = first_dfs(tree, child_i, goal)
        if match_i is not None:
            return match_i
        
    if is_goal(tree, i, goal):
        return i
    return None

## Class Assignment: Searching the maze using DFS. (10 minutes)

Remember 
* we have the four directions `UP`, `DOWN`, `LEFT`, `RIGHT`.
* we implemented `can_move(maze, me, direction)` and `on_my(me, direction)`.
* we know the start state with `find_start(maze)`.
* we save the list of explored points in `explored`.

In [ ]:
def solve_maze_dfs(maze, me, explored):
    # Search for `Maze.GOAL` and return the position.
    # 1. Begin with the start state.
    # 2. Check the latest node.
    #     * If it's the goal state.
    #     * If not, repeat 2 for each of its children.
    # 3. If all the children don't match the goal, return None.
    
    if me in explored:
        return None
    explored.add(me)
    
    r, c = me
    if maze[r][c] == Maze.GOAL:
        return r, c
    
    for direction in [UP, DOWN, LEFT, RIGHT]:
        if can_move(maze, me, direction):
            next_pos = on_my(me, direction)
            goal_pos = solve_maze_dfs(maze, next_pos, explored)
            # If by exploring in <direction> we found the goal.
            if goal_pos is not None:
                return goal_pos
    
    return None

In [ ]:
me = find_start(example_maze)
explored = set()
solve_maze_dfs(example_maze, me, explored)

In [ ]:
def solve_maze_dfs(maze):
    if maze.me in maze.explored:
        return False
    
    maze.explored.add(maze.me)
    maze.snapshot()
    
    if maze[maze.me] == Maze.GOAL:
        return True
    
    for direction in [LEFT, RIGHT, UP, DOWN]:
        if maze.to_my(direction) not in maze.explored:
            if maze[maze.to_my(direction)] != Maze.WALL:
#                 maze.drop_breadcrumb()
                maze.move(direction)
                maze.snapshot()

                if solve_maze_dfs(maze):
                    return True

                maze.move(opp_dir[direction])
#                 maze.pick_up_breadcrumb()
                maze.snapshot()
    
    return False

In [ ]:
example_maze.reset()
solve_maze_dfs(example_maze)
animate_maze(example_maze)

* DFS does not always return the most optimal solution.
* DFS may do more searching than necessary.

## Breadth-First Search (BFS)
![alt text](https://upload.wikimedia.org/wikipedia/commons/4/46/Animated_BFS.gif)

1. Begin with the start state.
2. Check the earliest node.
    * If it's the goal state, done!
    * If not, add its children to the tree.
3. Repeat 2 until the goal has been reached.

### Let's define the search order using a queue.

In [ ]:
from collections import deque

In [ ]:
queue = deque()

In [ ]:
queue.append('a')
queue

In [ ]:
queue.popleft()

In [ ]:
queue.append('b')
queue.append('c')
queue

### Class Assignment: Searching a maze using BFS. (15 minutes)

In [ ]:
def solve_maze_bfs(maze):
    # Search for `Maze.GOAL` and return the position.
    # 1. Begin with the start state.
    # 2. Check the earliest node.
    #     * If it's the goal state, done!
    #     * If not, add its children to the tree.
    # 3. Repeat 2 until the goal has been reached.
    # 
    # Hint: Use a queue to keep track of the earliest node.
    
    me = find_start(maze)
    explored = set()
    queue = dequeue()
    queue.append(me)
    
    while len(queue):
        me = queue.popleft()
        if me not in explored:
            explored.add(me)
            
            r, c = me
            if maze[r][c] == Maze.GOAL:
                return me

            for direction in [UP, DOWN, LEFT, RIGHT]:
                if can_move(maze, me, direction):
                    next_pos = on_my(me, direction)
                    queue.append(next_pos)
                    
    return None

In [ ]:
def solve_maze_bfs(maze):
    queue = deque([maze.start])
    while len(queue):
        pos = queue.popleft()
        maze.me = pos

        if maze.have_we_explored(maze.me):
            continue

        maze.explore(maze.me)
        maze.snapshot()

        if maze[maze.me] == Maze.GOAL:
            break

        for direction in [LEFT, RIGHT, UP, DOWN]:
            if not maze.have_we_explored(maze.to_my(direction)):
                if maze[maze.to_my(direction)] != Maze.WALL:
                    queue.append(maze.to_my(direction))

In [ ]:
example_maze.reset()
solve_maze_bfs(example_maze)
animate_maze(example_maze)

# Why DFS or BFS?

## DFS

In [ ]:
def solve_maze_dfs(maze):
    if maze.me in maze.explored:
        return False
    
    maze.explored.add(maze.me)
    maze.snapshot()
    
    if maze[maze.me] == Maze.GOAL:
        return True
    
    for direction in [LEFT, RIGHT, UP, DOWN]:
        if maze.to_my(direction) not in maze.explored:
            if maze[maze.to_my(direction)] != Maze.WALL:
#                 maze.drop_breadcrumb()
                maze.move(direction)
                maze.snapshot()

                if solve_maze_dfs(maze):
                    return True

                maze.move(opp_dir[direction])
#                 maze.pick_up_breadcrumb()
                maze.snapshot()
    
    return False

In [ ]:
example_maze.reset()
solve_maze_dfs(example_maze)
animate_maze(example_maze)

Pro:
* Does not have to search through the entire maze.

Con:
* Does not guarantee the optimal solution.

## BFS

In [ ]:
def solve_maze_bfs(maze):
    queue = deque([maze.start])
    while len(queue):
        pos = queue.popleft()
        maze.me = pos

        if maze.have_we_explored(maze.me):
            continue

        maze.explore(maze.me)
        maze.snapshot()

        if maze[maze.me] == Maze.GOAL:
            break

        for direction in [LEFT, RIGHT, UP, DOWN]:
            if not maze.have_we_explored(maze.to_my(direction)):
                if maze[maze.to_my(direction)] != Maze.WALL:
                    queue.append(maze.to_my(direction))

In [ ]:
example_maze.reset()
solve_maze_bfs(example_maze)
animate_maze(example_maze)

Pros
* Guarantees the optimal solution.

Cons:
* May need to search more states than necessary.

# A*  Search
Pronounced A-star search.

> Intuition: Search a promising direction deeply (like DFS), but can find the optimal solution like bfs.

## How do we estimate what the best next state is?
Let the current cost be $g(x)$, the estimated cost be $h(x)$.

The estimated cost is
$$f(x) = g(x) + h(x)$$

The true cost is
$$f^*(x) = g(x) + h^*(x)$$

For A* search, we want to make sure $h(x)$ is *admissible*: $h(x) \leq h^*(x)$. In other words, we must *underestimate* the true cost.

> 

## A* Search Algorithm
1. Get the state with the lowest estimated cost.
2. If it's the goal, we're done :)
3. Otherwise, for each of its children (next states), estimate their costs and add them in descending order.

# What is an admissible heuristic for a maze?

In [ ]:
example_maze.reset()
solve_maze_dfs(example_maze)
animate_maze(example_maze)

## Euclidean Distance
![alt text](media/euclidean.png)
This is admissible since the euclidean distance will always be shorter than the real path since we cannot travel diagonally.

### Class Assignment: Compute the euclidean distance of a given state. (5 minutes)

In [ ]:
import math

def find_goal(maze):
    for r, row in enumerate(maze):
        for c, val in enumerate(row):
            if val == Maze.GOAL:
                return r, c
    return None

def euclidean_distance(maze, me):
    x1, y1 = find_goal(maze)
    x2, y2 = me
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

In [ ]:
maze = example_maze.snapshots[24]
euclidean_distance(maze, maze.me)

## Manhattan Distance
![alt text](media/manhattan.png)
This is admissible since the number of steps in the best case is the manhattan distance. If there's no direct path, it'll cost more.
* Remember $h(x) \leq h^*(x)$ where $h(x)$ is the manhattan distance.

### Class Assignment: Compute the manhattan distance of a given state. (5 minutes)

In [ ]:
def manhattan_distance(maze, me):
    x1, y1 = find_goal(maze)
    x2, y2 = me
    return abs(x1 - x2) + abs(y1 - y2)

In [ ]:
maze = example_maze.snapshots[24]
manhattan_distance(maze, me)

## Let's use Python's priority queue to keep track of the lowest cost.

In [ ]:
from queue import PriorityQueue

In [ ]:
q = PriorityQueue()

We put items in a tuple `(cost, item)`.
* The queue will be sorted from lowest to highest cost.

In [ ]:
q.put((10, 'foo'))

In [ ]:
q.put((2, 'bar'))

In [ ]:
q.get()

In [ ]:
q.get()

In [ ]:
q.empty()

## Class Assignment: Solve the maze using A* Search

In [ ]:
def solve_maze_astar(maze):
    # Search for `Maze.GOAL` and return the position.
    # 1. Begin with the start state.
    # 2. Check the lowest cost node.
    #     * If it's the goal state, done!
    #     * If not, add its children to the tree.
    # 3. Repeat 2 until the goal has been reached.
    # 
    # Estimated cost = Cost to point + Heuristic cost
    # f(x) = g(x) + h(x)
    
    me = find_start(maze)
    queue = PriorityQueue()
    cost = manhattan_distance(maze, me)
    queue.put((cost, me))
    distance = dict() # g(x) = distance[pos]
    distance[me] = 0
    explored = set()
    
    while not queue.empty():
        _, pos = queue.get()
        me = pos
        
        if me in explored:
            continue
        explored.add(me)
        
        g = distance[me]
        for direction in [LEFT, RIGHT, UP, DOWN]:
            next_pos = on_my(me, direction)
            r, c = next_pos
            if maze[r][c] != 
        
    
    return 0, 0

In [ ]:
def astar_cost(maze, me):
    x1, y1 = me
    x2, y2 = maze.goal
    return abs(x1 - x2) + abs(y1 - y2)

def solve_maze_astar(maze):
    queue = PriorityQueue()
    queue.put((astar_cost(maze, maze.me), maze.me))
    distance = dict()
    distance[maze.me] = 0
    
    while not queue.empty():
        _, pos = queue.get()
        maze.me = pos

        if maze.have_we_explored(maze.me):
            continue

        maze.explore(maze.me)
        maze.snapshot()

        if maze[maze.me] == Maze.GOAL:
            break

        g = distance[maze.me]
        for direction in [LEFT, RIGHT, UP, DOWN]:
            next_pos = maze.to_my(direction)
            if not maze.have_we_explored(next_pos):
                if maze[next_pos] != Maze.WALL:
                    f = g + 1 + astar_cost(maze, next_pos)
                    distance[next_pos] = g + 1
#                     print(next_pos, f)
                    queue.put((f, next_pos))

In [ ]:
example_maze.reset()
solve_maze_astar(example_maze)
animate_maze(example_maze)